In [4]:
!pip install -q botometer

In [0]:
from openpyxl import load_workbook, Workbook
from botometer import Botometer
import pandas as pd
import random

In [11]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving config.csv to config.csv
User uploaded file "config.csv" with length 45 bytes


In [0]:
def print_dataset(dataset):
  for row in twitter_dataset.itertuples():
    print(row)
    # print(row[1])

def sort_dataset(dataset, col_names, ascending_vals):
  # sorting data frame by name
  # col_names.append('rank')
  # ascending_vals.append(True)
  dataset.sort_values(col_names, axis = 0, ascending = ascending_vals, 
                      inplace = True, na_position ='last')
  
def filter_dataset(dataset, col_name, col_value):
  # filtering data frame by name
  col_name = col_name.lower()
  if col_name in dataset.columns:
    return dataset[dataset[col_name].str.lower().isin([col_value.casefold()])]
  return dataset

In [0]:
def set_users_bot_prob(users_list):
  global users_bot_prob
  for user in users_list:
    if user not in users_bot_prob:
      bot_prob = random.uniform(0, 1)
      if bot_prob >= 0.6:
        users_bot_prob[user] = bot_prob

def discard_tweet_bots(dataset):
  global users_bot_prob
  set_users_bot_prob(dataset['author id'].unique())
  print(users_bot_prob)
  return dataset[~dataset['author id'].isin(users_bot_prob)]

def add_keyword_ranking(dataset):
  global keywords_data
  print(keywords_data)
  for keyword in keywords_data.itertuples():
    mask = (dataset['themes'].str.lower().str.contains(keyword[1].lower(), na=False))
    dataset['rank'][mask] = keyword[0]


In [0]:
users_bot_prob = {}
config_file2 = pd.read_csv('config.csv', header=None)
keywords_data = pd.read_csv('keywords.csv', header=None)
data_file = pd.ExcelFile('sol_sample_uhrcn.xlsx')

# Creating dataframe from Twitter data sheet
twitter_dataset = data_file.parse(data_file.sheet_names[0])
twitter_dataset.columns = twitter_dataset.columns.str.lower()
twitter_dataset['rank'] = ''
print('Initial shape:', twitter_dataset.shape)
# print(twitter_dataset.columns)

add_keyword_ranking(twitter_dataset)

twitter_dataset = discard_tweet_bots(twitter_dataset)
print('NO BOTS shape:', twitter_dataset.shape)
print('TOTAL BOTS:', len(users_bot_prob))
# Read the sorting commands from config file
list_sort_cols = []
list_sort_as_des = []
for command in config_file2.itertuples():
  if command[1].lower() == 'sortby':
    col_name = command[2].lower()
    if col_name in twitter_dataset.columns:
      list_sort_cols.append(col_name)
      list_sort_as_des.append(command[3].lower() == 'a')

if list_sort_cols:
  sort_dataset(twitter_dataset, list_sort_cols, list_sort_as_des)

# Process commands from config file
for command in config_file2.itertuples():
  if command[1].lower() != 'sortby':
    if command[1].lower() == 'filterby':
      twitter_dataset = filter_dataset(twitter_dataset, command[2], command[3])

# print(twitter_dataset.shape)

# Output file
twitter_dataset.to_excel("optimised_file.xlsx")


In [105]:
twitter_dataset

,source,host,link,date(et),time(et),localtime,category,author id,author name,author url,authority,followers,following,age,gender,language,country,province/state,city,location,sentiment,themes,classifications,entities,alexa rank,alexa reach,title,snippet,contents,summary,bio,unique id,post source,rank
6,TWITTER,http://www.twitter.com,http://twitter.com/carlogubi/statuses/11901089...,2019-10-31,23:30:49,31 Oct 2019 23:30:49,NaN,carlogubi,Carlo Gubitosa,http://twitter.com/carlogubi,8,7538,1922,NaN,MALE,Italian,Italy,NaN,NaN,Italy,NEGATIVE,politics;religion;,Person.religious_figure;Person.world_leader;Pe...,Vicario di Cristo;Papa;,NaN,NaN,NaN,Ai farisei xenofobi e ignoranti che predicano ...,Ai farisei xenofobi e ignoranti che predicano ...,NaN,"Giornalista, scrittore, autore di fumetti, pro...",1190108906783200000,IFTTT,10
79,TWITTER,http://www.twitter.com,http://twitter.com/carlogubi/statuses/11900634...,2019-10-31,20:30:22,31 Oct 2019 20:30:22,NaN,carlogubi,Carlo Gubitosa,http://twitter.com/carlogubi,8,7538,1922,NaN,MALE,Italian,Italy,NaN,NaN,Italy,NEGATIVE,NaN,NaN,NaN,NaN,NaN,NaN,"Queste si chiamano ""scienze sociali"". Ti dicon...","Queste si chiamano ""scienze sociali"". Ti dicon...","Queste si chiamano ""scienze sociali"". . L'igno...","Giornalista, scrittore, autore di fumetti, pro...",1190063497041920000,IFTTT,
90,TWITTER,http://www.twitter.com,http://twitter.com/nelloscavo/statuses/1190060...,2019-10-31,20:18:59,31 Oct 2019 20:18:59,NaN,nelloscavo,nello scavo,http://twitter.com/nelloscavo,8,14469,3250,NaN,NaN,Italian,Italy,NaN,NaN,Italia,NEGATIVE,NaN,Location.region;,Sardegna;,NaN,NaN,NaN,RT @AngiKappa: #eccidio #migranti L'orrore e l...,RT @AngiKappa: #eccidio #migranti L'orrore e l...,RT @AngiKappa: #eccidio #migranti L'orrore e l...,Avvenire journalist. Author (16 languages-70 c...,1190060630247689984,Twitter for Android,
27,TWITTER,http://www.twitter.com,http://twitter.com/Marinotoma/statuses/1190096...,2019-10-31,22:41:42,31 Oct 2019 22:41:42,NaN,Marinotoma,Marino Tomà 🇮🇹,http://twitter.com/marinotoma,7,2523,1395,NaN,MALE,Italian,Italy,NaN,NaN,Liguria (Italy),NEGATIVE,politics;,Person.politician;,Ministro;,NaN,NaN,NaN,RT @Drittorovescio_: Ecco il nostro assalto al...,RT @Drittorovescio_: Ecco il nostro assalto al...,RT @Drittorovescio_: Ecco il nostro assalto al...,Già Collaboratore Parlamentare Senato della Re...,1190096547935140096,Twitter for iPhone,2
25,TWITTER,http://www.twitter.com,http://twitter.com/LESOVICI/statuses/119009693...,2019-10-31,22:43:14,31 Oct 2019 22:43:14,NaN,LESOVICI,Monica,http://twitter.com/lesovici,6,1999,3763,NaN,FEMALE,Italian,Italy,lombardia,milano,"Milano, Lombardia",NEGATIVE,arts;politics;,Person.government_employee;Person.poet;,Il Principe;Le opere e i giorni;,NaN,NaN,NaN,RT @rottwi59: @stanzaselvaggia I vostri beniam...,RT @rottwi59: @stanzaselvaggia I vostri beniam...,RT @rottwi59: @stanzaselvaggia I vostri beniam...,NaN,1190096934981309952,Twitter Web App,2
33,TWITTER,http://www.twitter.com,http://twitter.com/giorgio12860698/statuses/11...,2019-10-31,22:19:11,31 Oct 2019 22:19:11,NaN,giorgio12860698,GIORGIO🇮🇹,http://twitter.com/giorgio12860698,6,2415,3257,NaN,MALE,Italian,Italy,NaN,NaN,Italia,NEGATIVE,politics;,Person.politician;,Ministro;,NaN,NaN,NaN,RT @Drittorovescio_: Ecco il nostro assalto al...,RT @Drittorovescio_: Ecco il nostro assalto al...,RT @Drittorovescio_: Ecco il nostro assalto al...,#italiano #sovranista #noIslam #noeuro #difesa...,1190090881082089984,Twitter for iPhone,2
58,TWITTER,http://www.twitter.com,http://twitter.com/moriggi/statuses/1190076544...,2019-10-31,21:22:13,31 Oct 2019 21:22:13,NaN,moriggi,LUISA 🇮🇹🇮🇹🇮🇹💚🇮🇹🇮🇹🇮🇹,http://twitter.com/moriggi,6,681,37,NaN,FEMALE,Italian,Italy,NaN,NaN,"Lombardia, Italia",NEGATIVE,politics;,Org.nonprofit;Org.government_agency;,Alto commissariato delle Nazioni Unite per i r...,NaN,NaN,NaN,RT @Kwizera06428241: Per l'#UNHCR non tutti i ...,RT @Kwizera06428241: Per l'#UNHCR non tutti i ...,RT @Kwizera06428241: Per l'#UNHCR non t